In [8]:
!pip install langchain_community langchain_text_splitters langchain_chroma sentence-transformers gradio groq dotenv pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 14.9 MB/s eta 0:00:00


In [9]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from uuid import uuid4
import os
import gradio as gr
from pathlib import Path
from groq import Groq
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer


In [10]:
load_dotenv()

# Constants
DATA_PATH = "data"
CHROMA_PATH = "chroma_db"
KEY = os.getenv("GROQ_API_KEY")


# Initialize Groq client
client = Groq(
    api_key=KEY
)



In [11]:
class NomicEmbedEmbeddings(Embeddings):
    def __init__(self, model_name: str = "nomic-ai/nomic-embed-text-v1.5"):
        self.model = SentenceTransformer(model_name, trust_remote_code=True)

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        # Prefix each document with 'search_document:'
        prefixed_texts = [f"search_document: {text}" for text in texts]
        return self.model.encode(prefixed_texts, normalize_embeddings=True).tolist()

    def embed_query(self, text: str) -> list[float]:
        # Prefix the query with 'search_query:'
        prefixed_text = f"search_query: {text}"
        return self.model.encode([prefixed_text], normalize_embeddings=True)[0].tolist()


embedding_model = NomicEmbedEmbeddings()
vector_store = Chroma(
    collection_name="lecture_notes",
    embedding_function=embedding_model,
    persist_directory=CHROMA_PATH,
)

In [12]:
# Load and split PDFs
if not os.path.exists(DATA_PATH):
    from google.colab import drive
    drive.mount('/content/drive')

    # Source and destination paths
    SOURCE_PATH = "/content/drive/MyDrive/mistral"
    DEST_PATH = Path(DATA_PATH)
    DEST_PATH.mkdir(parents=True, exist_ok=True)

    # Copy all PDFs from Google Drive
    for file_name in os.listdir(SOURCE_PATH):
        if file_name.endswith(".pdf"):
            src = os.path.join(SOURCE_PATH, file_name)
            dst = os.path.join(DATA_PATH, file_name)
            os.system(f'cp "{src}" "{dst}"')

    print("PDFs copied from Google Drive to 'data/' directory.")
else:
    print("'data/' directory already exists. Skipping Google Drive copy.")

loader = PyPDFDirectoryLoader(DATA_PATH)
raw_documents = loader.load()

'data/' directory already exists. Skipping Google Drive copy.


In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
)

chunks = text_splitter.split_documents(raw_documents)

uuids = [str(uuid4()) for _ in range(len(chunks))]
vector_store.add_documents(documents=chunks, ids=uuids)

# Set up retriever
retriever = vector_store.as_retriever(search_kwargs={'k': 5})

In [16]:
# Chat function using Groq LLaMA-4
def answer_question(query, history):

    docs = retriever.invoke(query)
    knowledge = "\n\n".join([doc.page_content for doc in docs])

    prompt = f"""[INST]
You are a helpful assistant answering questions based **only** on the "Relevant Lecture Notes" provided below.

- Do not answer using your own knowledge or assumptions.
- If the answer is not found in the content, reply: "The answer is not available in the lecture notes."
- Keep your answers short, clear, and relevant to the question.
- If a list or step-by-step format is appropriate, use it.
- Refer to the provided chat history rather than your internal one.

Question: {query}

Relevant Lecture Notes: {knowledge}

Conversation history: {history}

[/INST]
"""

    try:
        # Send prompt to Groq
        completion = client.chat.completions.create(
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0.01,
            max_tokens=2048,
            top_p=0.1,
            stream=True
        )

        # Stream response content
        full_response = ""
        for chunk in completion:
            full_response += chunk.choices[0].delta.content or ""


        return full_response.strip()

    except Exception as e:
        return f"Error: {e}"

# Launch Gradio chatbot
chatbot = gr.ChatInterface(
    fn=answer_question,
    textbox=gr.Textbox(
        placeholder="Ask a question from CTSE lectures...",
        container=False
    ),
)
chatbot.launch()


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://59e2d3f6f29fa36c17.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
